# Сборный проект по первому блоку курса Data Science

# 1. Введение

Вы работаете в интернет-магазине «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Вам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

### Описание данных
- Name — название игры
- Platform — платформа
- Year_of_Release — год выпуска
- Genre — жанр игры
- NA_sales — продажи в Северной Америке (миллионы проданных копий)
- EU_sales — продажи в Европе (миллионы проданных копий)
- JP_sales — продажи в Японии (миллионы проданных копий)
- Other_sales — продажи в других странах (миллионы проданных копий)
- Critic_Score — оценка критиков (максимум 100)
- User_Score — оценка пользователей (максимум 10)
- Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

# 2. Открытие файла с данными и изучение общей информации

In [146]:
import pandas as pd
from scipy import stats as st
import numpy as np
import matplotlib.pyplot as plt

In [147]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/games.csv')
display(df.head())
df.info()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


Источник данных представляет собой таблицу из 11 колонок и 16715 строк. Содержание колонок в целом соответствует описанию данных из Введения.

# 3. Подготовка данных

## 3.1 Замена названий столбцов (приведение к нижнему регистру)

In [148]:
df.columns = map(lambda x:x.lower(),df.columns)
df.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


Названия столбцов переведены в нижний регистр

## 3.2 Преобразование данных в нужные типы

In [149]:
import math
def is_whole_number(number):
    return math.floor(number) == number
def all_whole_numbers(numbers):
    return all(map(is_whole_number, numbers))
df_temp = df['critic_score'].dropna()
print(all_whole_numbers(df_temp))

True


Проверил что в колонке **critic_score** все валидные значения не имеют дробной части и предсатвляют собой целое число процентов c нулем после десятичной точки.

In [150]:
print('Количество записей в таблице с значенем user_score == tbd: ', df[df['user_score'] == 'tbd']['user_score'].count())
print('Количество записей в таблице с значенем user_score == 0: ', df[df['user_score'] == '0']['user_score'].count())
df.loc[df['user_score'] == 'tbd','user_score'] = '0'
print('Количество записей в таблице с значенем user_score == tbd: ', df[df['user_score'] == 'tbd']['user_score'].count())
print('Количество записей в таблице с значенем user_score == 0: ', df[df['user_score'] == '0']['user_score'].count())


Количество записей в таблице с значенем user_score == tbd:  2424
Количество записей в таблице с значенем user_score == 0:  1
Количество записей в таблице с значенем user_score == tbd:  0
Количество записей в таблице с значенем user_score == 0:  2425


Посчитал количество записей в таблице с значенем user_score равным tbd и 0

In [151]:
df.info()
display(df.head())
for col in ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']:
    df[col] = df[col] * 1000000
df = df.astype({"name":'string',"platform":'string',"year_of_release":'Int64',"genre":'string',"na_sales":'int',"eu_sales":'int',"jp_sales":'int',"other_sales":'int',"critic_score":'Int64',"rating":'string'}, errors = 'ignore')
df['user_score'] = pd.to_numeric(df['user_score'], errors = 'coerce')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  string 
 1   platform         16715 non-null  string 
 2   year_of_release  16446 non-null  Int64  
 3   genre            16713 non-null  string 
 4   na_sales         16715 non-null  int64  
 5   eu_sales         16715 non-null  int64  
 6   jp_sales         16715 non-null  int64  
 7   other_sales      16715 non-null  int64  
 8   critic_score     8137 non-null   Int64  
 9   user_score       10014 non-null  float64
 10  rating           9949 non-null   string 
dtypes: Int64(2), float64(1), int64(4), string(4)
memory usage: 1.4 MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41360000,28960000,3770000,8450000,76,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29080000,3580000,6810000,770000,<NA>,NaN,<NA>
2,Mario Kart Wii,Wii,2008,Racing,15680000,12760000,3790000,3290000,82,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15610000,10930000,3280000,2950000,80,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11270000,8890000,10220000,1000000,<NA>,NaN,<NA>


Были проведены следующие изменения типов:
- колонки **name** , **platform**, **genre**, **rating** переводятся в **string** так как они содержат строковые значения (или NaN)
- колонка **year_of_release** содержит год релиза который представляет из себя целое число, я использовал приведение к dtype Int64, при котором произошло и отбрасывание дробной части в виде 0
- колонки **na_sales**, **eu_sales**, **jp_sales**, **other_sales** содержат число миллионов с дробной частью - я перевел сначала их просто в число продаж путем умножения на 1 000 000 а потом привел к целому - так как использование чисел типа float в Python сопряжено с накопленной ошибкой вычисления и по возможности если можно их надо избегать. В данном случае именно такой случай.
- колонка **critic_score** содержит рейтинг от критиков в интервале от 0 до 100, который не содержит дробной части после запятой (выше я провел проверку этого факта). Таким образом, эта колонка содержит целое число процентов и ее тип можно перевести в int с отбрасыванием дробной части
Для преобразования типов я использовал режим **errors = 'ignore'** который в случае ошибки оставлял исходное значения - в даном случае все NaN были заменены на <NA> которые далее могут быть обнаружены и удалены в случе необходимости
    
Если сравнить выводы **df.info()** до и после преобразования типов, то можно увидеть, что в некоторых колонках нет пропусков и число значений - общему числу строк = 16715. Для ряда колонок число значений меньше и это число не изменилось после изменения типов колонки - за исключением колонки **user_score** где помимо значений NaN пристутсвует еще и ряд строк с строковым значеним = tbd, которое не может быть преобразовано в float и таких строк довольно большое количество = 2424 то есть их нежелательно просто отбросить. 
Колонка **user_score** содержит рейтинг пользователя в интервале от 0 до 10, который может содержать дробную часть - поэтому я ее тип надо перевести в float, используя метод to_numeric, но для того, чтобы избежать ошибок парсинга пердварительно я заменю все значения **tbd** на строковое значение 0 а потом преобразованием типа перевел его в 0 типа float. При этом я подсчитал количество записей с изначальным значением **user_score** == 0 и это оно = 1 - то есть можно эти данные проигнорировать и "слить" их в одну группу.

## 3.3 Обработка пропусков

In [152]:
df[df['user_score'] == 0]['user_score'].count()

2425

- Объясните, почему заполнили пропуски определённым образом или почему не стали это делать

Общее количество строк в dataframe равно 16715 но ряд столбцов имеет пропуски и меньшее число значимых величин а именно:
    - name - 2 пропуска
    - year_of_release - 269 пропусков
    - genre - 2 пропуска
    - critic_score - 8578 пропусков
    - user_score - 6701 пропусков и 2424 значения **tbd** которые не могут быть проеобразованы в float и они были установлены в 0
    - rating - 6766 пропусков
    
Пропуски я оставил после преобразования типов как есть (NaN или \<NA\>), значение 'tbd' для колонки **user_score** установил в 0 (подробно описал в предыдущем пункте 3.2)
- Опишите причины, которые могли привести к пропускам
    - Для полей **name**, **genre** - ошибки ввода - например при заполнении формы. Предлагаю их оставить как есть так как их количество очень мало и ими можно пренебречь при рассчете характеристик.
    - Поле **year_of_release** изначально было числовым и там исключено наличие нечисловых данных - пропуск может быть связан либо с ошибкой при вводе даных - например формат ввода даты не соответствовал предложенному шаблону.
- Обратите внимание на аббревиатуру 'tbd' в столбце с оценкой пользователей. Отдельно разберите это значение и опишите, как его обработать
    Уже объяснено в п 3.2

## 3.4 Подсчет суммарных продаж во всех регионах

# 4. Провести исследовательский анализ данных

## 4.1 Распределение игр по годам

## 4.2 Распределение продаж по платформам

## 4.3 Данные за актуальный период

## 4.4 Удаление данных за предыдущие годы

## 4.5 Выбор потенциально прибыльных платформ

## 4.6 Построение графика «ящик с усами» по глобальным продажам игр в разбивке по платформам

## 4.7 Анализ влияния на продажи внутри одной популярной платформы отзывов пользователей и критиков

## 4.8 Сравнение выводов с продажами игр на других платформах

## 4.9 Анализ общего распределения игр по жанрам

# 5. Составление портрета пользователя каждого региона

## 5.1 Определение самых популярных платформ (top 5)

## 5.2 Определение самых популярных жанров (top 5)

## 5.3 Влияние рейтинга ESRB на продажи в отдельном регионе

# 6. Проверка гипотез

## 6.1 Средние пользовательские рейтинги платформ Xbox One и PC одинаковые

## 6.2 Средние пользовательские рейтинги жанров Action и Sports разные

# 7. Выводы

<div style="border:dashed green 1px; background:#097482; padding: 20px">
<span style="color:white">Мои комментарии version:</span>

Я хотел бы привести тут формат оформления ноутбука, который я буду использовать:

-  Введение
-  Шаги проекта оформленные как markdown header 1го уровня - например **2.Предобработка данных**
-  Внутри каждого шага промежуточные задачи как заголовки 2го уровня (если они есть)- например 
   - **2.1 Приведите столбец date к типу даты pandas**
   - **2.2 Создайте новый столбец с номером месяца на основе столбца date**
   - **2.3 Проверьте наличие пропущенных значений и дубликатов в датафреймах.** 
- Сначала идет заголовок 1го уровня в ячейке Markdown, потом - если есть - заголовок второго уровня в ячейке Markdown, потом код на Python, потом описание этого шага в ячейке Markdown
- Выводы - включают в себя результаты по каждому пункту и общий вывод в виде рекомендаций для бизнеса.Посчитайте суммарные продажи во всех регионах

Все свои комментарии в ответ на комментарии ревьюера я буду далее добавлять, используя этот стиль